<center>
<h1 style="font-family:verdana">
 💻 🧑 Reconeixement d'entitats anomenades 🧑 💻


<p> 🎯 <b>Objectiu</b>: en aquesta segona part de la pràctica aprendrem a reconèixer entitats anomenades, és a dir, identificar i classificar entitats en una oració que poden ser (en el context de reserva de vols) la ciutat de sortida o d'arribada, la data del mes i el dia, si es de negocis o turista o altres categories específiques. Aquesta tasca se sol plantejar etiquetant cada paraula amb una etiqueta de la categoria de l'entitat a la qual correspon.

Tant la classificació d'intencions (tasca estudiada a la primera part) com el reconeixement d'entitats anomenades (Name Entity Recognition, NER) que veurem en aquesta part són components crucials dels sistemes de processament del llenguatge natural (Natural Language Processing, NLP) i sovint s'utilitzen junts per crear aplicacions d'intel·ligència artificial (IA) conversacionals més sofisticades. Per exemple, en un xatbot, la classificació d'intencions ajuda a entendre la intenció principal de l'usuari, mentre que el reconeixement d'entitats anomenades ajuda a extreure entitats rellevants per proporcionar respostes més contextualitzades.


<p> ✨ <b>Contingut</b>: els passos d'aquesta segona part seran similars a la primera. En primer lloc, analitzarem el conjunt de dades. En segon lloc, prepararem les dades per a poder entrenar el model. I finalment dissenyarem l'arquitectura i entrenarem el model. </p>


<p> ✏ <b>Exercicis</b>: en cada secció anireu trobant exercicis que haureu d'anar resolent. </p>

---

<h2> Índex </h2>

1. [Inspecció del conjunt de dades](#section-one)
  * [Exercici 1](#ex-one)
  * [Exercici 2](#ex-two)
2. [Preprocessament de dades](#section-two)
  * [Exercici 3](#ex-three)
  * [Exercici 4](#ex-four)
3. [Disseny del model i entrenament](#section-three)
  * [Exercici 5](#ex-five)
  * [Exercici 6](#ex-six)
4. [Lliurable](#section-four)

---

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report


KeyboardInterrupt: 

<h1><a name="section-one"> 1. Inspecció del conjunt de dades </a></h1>

In [ ]:
'''!pip install gdown
!gdown "https://drive.google.com/uc?id=1u2wzXvsuscLeFHwXcDwMDaNDy0u_99-t"
!tar -zxf nlu_ATIS_data.tar.gz'''

'!pip install gdown\n!gdown "https://drive.google.com/uc?id=1u2wzXvsuscLeFHwXcDwMDaNDy0u_99-t"\n!tar -zxf nlu_ATIS_data.tar.gz'

In [ ]:
!ls data

"ls" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


En aquesta segona part de la pràctica utilitzarem el mateix dataset que a la primera part, però en aquest cas utilitzarem la **primera columna** que correspon a les **oracions** introduïdes pels usuaris i la **segona columna** on trobarem les oracions d'entrada en format **BILOU**.

El format BILOU  és un esquema d'etiquetatge que es fa servir a les tasques de reconeixement d'entitats anomenades. El nom *BILOU* representa les etiquetes utilitzades en aquest esquema:

*   **B** - Beginning: primer token d'una entitat.
*   **I** - Inside: token dins d'una entitat.
*   **L** - Last: últim token de l'entitat.
*   **O** - Outside: token que no pertany a cap entitat.
*   **U** - Unit: entitats d'un sol token

A continuació, carregarem les dades per visualitzar i poder entendre millor aquestes etiquetes.

---

 <h1><a name="ex-one"><center> ✏ Exercici 1 ✏</a></h1>

A continuació us demanem que carregueu els dos CSVs de la carpeta `data`: `train.csv`, `test.csv` utilitzant pandas. Recorda que aquests CSVs no tenen capçalera. Agafeu les 900 últimes lineas del fitxer `train.csv` per crear un dataframe per validació.

In [ ]:
#TODO
train_data = pd.read_csv('data/train.csv', header=None)
val_data = train_data.tail(900)
train_data = train_data.head(len(train_data) - 900)
test_data = pd.read_csv('data/test.csv', header=None)

print('Training size:', len(train_data))
print('Validation dataset size:', len(val_data))
print('Test dataset size:', len(test_data))

# Limpiar comillas simples y dobles de todas las columnas
train_data = train_data.map(lambda x: x.replace("'", "").replace('"', ""))
test_data = test_data.map(lambda x: x.replace("'", "").replace('"', ""))
val_data = val_data.map(lambda x: x.replace("'", "").replace('"', ""))

Training size: 4078
Validation dataset size: 900
Test dataset size: 893


---

 <h1><a name="ex-two"><center> ✏ Exercici 2 ✏</a></h1>

Tal com hem introduït abans, per a aquest exercici ens centrarem en la **primera** i la **segona** columna. Per tant, ara us demanem que guardeu en les següents variables, les llistes corresponents a les oracions i a les etiquetes de les tres particions (`train`, `validation` i `test`).

In [ ]:
#TODO
# Forma COMPACTA - Seleccionar columnas específicas directamente
train_sentences = train_data[0].tolist()  # Columna 0
train_labels = train_data[1].tolist()      # Columna 1

val_sentences = val_data[0].tolist()
val_labels = val_data[1].tolist()

test_sentences = test_data[0].tolist()
test_labels = test_data[1].tolist()


In [ ]:
test_labels

[' O O O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O O O O O B-stoploc.city_name I-stoploc.city_name',
 ' O B-depart_date.month_name B-depart_date.day_number O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_time.time_relative B-depart_time.time I-depart_time.time',
 ' O B-depart_date.month_name B-depart_date.day_number O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name',
 ' O O O O O O B-round_trip I-round_trip O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_date.month_name B-depart_date.day_number',
 ' O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name I-toloc.city_name O B-depart_date.month_name B-depart_date.day_number O B-airline_name I-airline_name',
 ' O O O O O B-fromloc.city_name O B-toloc.city_name B-round_trip I-round_trip O B-depart_date.day_name B-depart_time.period_of_day O B-depart_date.day_name B-depart_time.period_of_day',
 ' B-depart_date.day_name B-depa

---

Si tot ha anat bé ja podem analitzar quin aspecte té el format BILOU.

In [ ]:
print(train_sentences[5])
print(train_labels[5])

i need a flight tomorrow from columbus to minneapolis
 O O O O B-depart_date.today_relative O B-fromloc.city_name O B-toloc.city_name


Aquest hauria de ser el resultat obtingut si executeu la cel·la anterior:

```


i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
 "O O O O O B-fromloc.city_name O B-depart_time.time I-depart_time.time O O O B-toloc.city_name O B-arrive_time.time O O B-arrive_time.period_of_day"


```

En aquest exemple, `boston` i `838 am` corresponen a l'entitat "ciutat de sortida" i a l'entitat "hora de sortida" respectivament (`fromloc.city_name`,`depart_time.time`). `838` correspon al primer token que pertany la entitat `depart_time.time` i `am` al segon de la mateixa entitat. Els tokens com `i` o `want` no pertanyen a cap entitat.

In [ ]:
print(train_sentences[1])
print(train_labels[1])

what flights are available from pittsburgh to baltimore on thursday morning
 O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day


I aquest hauria de ser el resultat de la cel·la anterior:

```
what flights are available from pittsburgh to baltimore on thursday morning
 O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day
```

En l'anterior exemple, `what`, `flights`, `are`, `available`, `from`, `to` i `on` estan etiquetades com a no pertanyents a cap entitat. I, en canvi, `pittsburgh` pertany a l'entitat d'un sol token anomenada ciutat d'arribada (`toloc.city_name`). També hi ha entitats compostes com `baltimore` que pertany a l'entitat ciutat de sortida (`fromloc.city_name`) i `thursday morning` que pertanyen a les entitats data de sortida (`depart_date.day_name`, `depart_time.period_of_day`).

Vegem-ne la llista completa d'entitats.

In [ ]:
## Problema desbalanceig de les classes (hi ha masses classes)

## Predicció Seq2Seq

from collections import Counter

def count_unique_entities(list_of_label_sentences):
  flat_labels = []
  for labels in list_of_label_sentences:
    flat_labels += labels.split()
  c = Counter(flat_labels)
  return len(c), list(c.keys())

num_unique_entities, unique_entities = count_unique_entities(train_labels)

print("Number of different entities:", num_unique_entities)

print("Unique entities:", unique_entities)

Number of different entities: 119
Unique entities: ['O', 'B-fromloc.city_name', 'B-depart_time.time', 'I-depart_time.time', 'B-toloc.city_name', 'B-arrive_time.time', 'B-arrive_time.period_of_day', 'B-depart_date.day_name', 'B-depart_time.period_of_day', 'B-flight_time', 'I-flight_time', 'I-fromloc.city_name', 'B-cost_relative', 'B-round_trip', 'I-round_trip', 'B-fare_amount', 'I-fare_amount', 'B-depart_date.today_relative', 'I-toloc.city_name', 'B-city_name', 'B-stoploc.city_name', 'B-toloc.airport_code', 'B-depart_time.time_relative', 'B-class_type', 'I-class_type', 'B-depart_date.date_relative', 'B-airline_name', 'I-airline_name', 'B-arrive_time.time_relative', 'B-depart_time.start_time', 'I-depart_time.start_time', 'B-depart_time.end_time', 'I-depart_time.end_time', 'B-fromloc.airport_name', 'I-fromloc.airport_name', 'B-toloc.state_name', 'B-depart_date.day_number', 'I-depart_date.day_number', 'B-depart_date.month_name', 'B-mod', 'B-fare_basis_code', 'B-transport_type', 'B-flight_m

<h1><a name="section-two"> 2. Preprocessament de dades </a></h1>

El processament de les dades serà semblant al de la primera part d'aquesta pràctica.

---

 <h1><a name="ex-three"><center> ✏ Exercici 3 ✏</a></h1>

En aquest exercici us demanem que realitzeu els passos següents per preparar les dades.

 1. El primer pas serà construir el vocabulari a partir de les paraules presents a les oracions d'entrenament.

2. El segon pas serà convertir les oracions en seqüències de nombres enters usant el tokenitzador.

3. El tercer pas serà guardar la longitud original de cada oració. Això ens serà útil per evaluar el nostre model sense tenir en compte el padding.

3. Finalment, per aconseguir que totes les seqüències tinguen la mateixa longitud, fixarem la longitud segons la màxima trobada a l'entrenament i afegirem zeros a les oracions de menor longitud.

Recordeu que podeu consultar com fer-ho a la documentació de la llibrería:
* <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer"> Tokenizer </a>
* <a href="https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences"> Pad Sequences </a>




In [ ]:
#TODO
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index 

train_sequences = tokenizer.texts_to_sequences(train_sentences)
max_sequence_length = max(len(seq) for seq in train_sequences)

# Guardar longitudes ORIGINALES de cada secuencia (antes del padding)
train_sequence_lengths = [len(seq) for seq in train_sequences]
train_pad_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')

val_sequences = tokenizer.texts_to_sequences(val_sentences)
val_sequence_lengths = [len(seq) for seq in val_sequences]
val_pad_sequences = pad_sequences(val_sequences, maxlen=max_sequence_length, padding='post')

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_sequence_lengths = [len(seq) for seq in test_sequences]
test_pad_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

print(f"📊 Secuencias procesadas:")
print(f"Train: {len(train_sequences)} oraciones, max_length={max_sequence_length}")
print(f"Val: {len(val_sequences)} oraciones")
print(f"Test: {len(test_sequences)} oraciones")

📊 Secuencias procesadas:
Train: 4078 oraciones, max_length=46
Val: 900 oraciones
Test: 893 oraciones


In [ ]:
train_pad_sequences[0]

array([ 13,  70,   2,  39,   3,  10,  65, 416,  85,  18,  76,  17,  14,
        65, 493,  17,   5,  37,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])


---



Tal com vam fer a la primera part, aquí també hem de convertir les diferents classes d'entitats en vectors one-hot.

---

 <h1><a name="ex-four"><center> ✏ Exercici 4 ✏</a></h1>

Per aconseguir-ho haureu de seguir els passos següents.

 1. En primer lloc, haureu d'esbrinar quantes etiquetes diferents hi ha. Podeu prendre com a exemple la funció `count_unique_entities` per fer-ho. Tingueu en compte que haureu de modificar la funció, ja que per exemple aquesta funció considera que `B-depart_time.time`, `I-depart_time.time`, `L-depart_time.time`, `U-depart_time.time` són la mateixa entitat. En aquest exercici, necessitarem comptar-les per separat. També l'entitat O ha de ser considerada com una classe.

 2. El segon pas serà codificar les diferents classes trobades en etiquetes numèriques. Tingueu en compte que cada paraula de l'oració té una etiqueta i, per tant, per a cada oració tindrem una llista d'etiquetes. El *padding* el codificarem amb l'etiqueta corresponent a **O** (outside). Per ajudar-vos hem preparat el processament per a les etiquetes d'entrenament, intenteu comprendre el que es fa i així repetir-ho per a la partició de validació i test.

 3. Finalment, haureu de convertir les diferents classes a vectors one-hot. Recordeu de nou que per a cada oració tindrem una llista de vectors one-hot.


 Podeu consultar els apartats de la documentació:
 * <a href=https://www.tensorflow.org/guide/keras/understanding_masking_and_padding> Masking and Padding </a>

 * <a href=https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical> To Categorical </a>



In [ ]:
type(train_labels)

list

In [ ]:
test_labels

[' O O O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O O O O O B-stoploc.city_name I-stoploc.city_name',
 ' O B-depart_date.month_name B-depart_date.day_number O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_time.time_relative B-depart_time.time I-depart_time.time',
 ' O B-depart_date.month_name B-depart_date.day_number O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name',
 ' O O O O O O B-round_trip I-round_trip O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_date.month_name B-depart_date.day_number',
 ' O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name I-toloc.city_name O B-depart_date.month_name B-depart_date.day_number O B-airline_name I-airline_name',
 ' O O O O O B-fromloc.city_name O B-toloc.city_name B-round_trip I-round_trip O B-depart_date.day_name B-depart_time.period_of_day O B-depart_date.day_name B-depart_time.period_of_day',
 ' B-depart_date.day_name B-depa

In [ ]:
#TODO
#Step 1
def count_unique_entities(list_of_label_sentences):
  flat_labels = []
  for labels in list_of_label_sentences:
    flat_labels += labels.split()
  unique_entities = Counter(flat_labels)
  return len(unique_entities), unique_entities

num_unique_entities, unique_entities = count_unique_entities(train_labels)
print("-------------Train labels exploration------------")
print("Number of different entities:", num_unique_entities)
print("Unique entities:", unique_entities)

#Step 2
label_encoder = LabelEncoder()
label_encoder.fit(list(unique_entities.keys()))
train_numerical_labels = [label_encoder.transform(labels.split()).tolist() for labels in train_labels]
train_pad_labels = pad_sequences(train_numerical_labels, maxlen=max_sequence_length, padding='post', value=label_encoder.transform(['O']))

def remove_sentences(list_labels, list_sequences):
  idx_to_remove = []
  labels_to_remove = set()
  for idx, labels in enumerate(list_labels):
    for label in labels.split():
      if label not in unique_entities:
        idx_to_remove.append(idx)
        labels_to_remove.add(label)

  labels = [elem for i, elem in enumerate(list_labels) if i not in idx_to_remove]
  sequences = [elem for i, elem in enumerate(list_sequences) if i not in idx_to_remove]
  print("Original size:", len(list_labels))
  print("Removed sentences:", len(idx_to_remove))
  print("Labels removed:", set(labels_to_remove))
  return labels, np.array(sequences)

print("-------------Test labels codification------------")
test_labels_cleaned, test_pad_sequences = remove_sentences(test_labels, test_pad_sequences)
test_numerical_labels = [label_encoder.transform(labels.split()).tolist() for labels in test_labels_cleaned]
test_pad_labels = pad_sequences(test_numerical_labels, maxlen=max_sequence_length, padding='post', value=label_encoder.transform(['O']))

print("-------------Validation labels codification------------")
val_labels_cleaned, val_pad_sequences = remove_sentences(val_labels, val_pad_sequences)
val_numerical_labels = [label_encoder.transform(labels.split()).tolist() for labels in val_labels_cleaned]
val_pad_labels = pad_sequences(val_numerical_labels, maxlen=max_sequence_length, padding='post', value=label_encoder.transform(['O']))

#Step 3
train_labels_one_hot = to_categorical(train_pad_labels, num_classes=len(label_encoder.classes_))
test_labels_one_hot = to_categorical(test_pad_labels, num_classes=len(label_encoder.classes_))
val_labels_one_hot = to_categorical(val_pad_labels, num_classes=len(label_encoder.classes_))
print("-------------One-hot encoding shapes------------")
print("Train labels one-hot shape:", train_labels_one_hot.shape)
print("Test labels one-hot shape:", test_labels_one_hot.shape)
print("Validation labels one-hot shape:", val_labels_one_hot.shape)
print("-------------Sequence shapes------------")
print("Train sequences shape:", train_pad_sequences.shape)
print("Test sequences shape:", test_pad_sequences.shape)
print("Validation sequences shape:", val_pad_sequences.shape)

-------------Train labels exploration------------
Number of different entities: 119
Unique entities: Counter({'O': 29586, 'B-toloc.city_name': 3571, 'B-fromloc.city_name': 3549, 'I-toloc.city_name': 898, 'B-depart_date.day_name': 745, 'B-airline_name': 575, 'I-fromloc.city_name': 564, 'B-depart_time.period_of_day': 491, 'I-airline_name': 352, 'B-depart_date.day_number': 316, 'B-depart_time.time': 307, 'B-depart_date.month_name': 303, 'B-round_trip': 275, 'B-flight_mod': 275, 'B-cost_relative': 273, 'B-depart_time.time_relative': 271, 'I-round_trip': 268, 'I-depart_time.time': 249, 'B-stoploc.city_name': 205, 'B-arrive_time.time': 178, 'B-city_name': 177, 'B-class_type': 177, 'B-arrive_time.time_relative': 161, 'I-class_type': 148, 'B-flight_stop': 140, 'I-arrive_time.time': 139, 'B-airline_code': 114, 'I-depart_date.day_number': 96, 'I-fromloc.airport_name': 88, 'B-toloc.state_code': 71, 'B-arrive_date.day_name': 70, 'B-depart_date.today_relative': 66, 'B-fromloc.airport_name': 66, 'B-

In [ ]:
train_labels_one_hot

array([[[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0.

<h1><a name="section-three"> 3. Disseny del model i entrenament </a></h1>

---
<h1><a name="ex-five"><center> ✏ Exercici 5 ✏</a></h1>

De forma similar com féreu a la primera part de la pràctica us demanem que dissenyeu l'arquitectura i entreneu el model. Podeu fer servir una arquitectura similar. Ara bé, per capturar les dependències seqüencials podeu canviar la capa de GlobalMaxPooling1D per una LSTM.

In [ ]:
s = train_pad_labels.shape
train_flat_labels = train_pad_labels.reshape(s[0]*s[1])
train_unq_labels = np.unique(train_flat_labels)
print(train_flat_labels.shape, train_unq_labels)

(187588,) [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118]


In [ ]:
## Decoder Transformer
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = keras.layers.MultiHeadAttention(      ## NO MASSA HEADS PERQUÈ ES FIXARAN EN ALTRES PROPIETATS QUE DISTORCIONEN NER
            num_heads=num_heads, key_dim=embed_dim
        )
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(ff_dim, activation="relu"),  # PROJECCIÓ A DIM MÉS ALT (FF_DIM)
                keras.layers.Dense(embed_dim), # TORNAR A DIM EMBED_DIM
            ]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    ## Forward
    def call(self, inputs, training=False):  # training false pel test
        attn_output = self.att(inputs, inputs) 
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # layer(add)
        ## NO CROSS-ATTENTION PQ NO ENCODER
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        # MAXLEN = MÀXIMA LLARGADA DE LA SEQUÈNCIA

    def call(self, inputs):
        maxlen = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)  # GENERAR ÍNDEX DE POSICIONS
        position_embeddings = self.pos_emb(positions) # EMBEDDING DE LES POSICIONS
        token_embeddings = self.token_emb(inputs) # EMBEDDING DELS TOKENS
        return token_embeddings + position_embeddings





In [ ]:
#TODO
embedding_dim = 256
vocab_size = len(tokenizer.word_index) + 1  # Usar el vocab real del tokenizer

model = Sequential()
model.add(TokenAndPositionEmbedding(maxlen=max_sequence_length, vocab_size=vocab_size, embed_dim=embedding_dim))
model.add(TransformerBlock(embed_dim=embedding_dim, num_heads=4, ff_dim=256))
model.add(keras.layers.Dense(len(unique_entities), activation="softmax"))  # Output para cada token
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])  # categorical para one-hot

# Train the model
batch_size = 32
epochs = 10
history = model.fit(train_pad_sequences, train_labels_one_hot, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=(val_pad_sequences, val_labels_one_hot))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_pad_sequences, test_labels_one_hot, batch_size=batch_size)
print(f"\nTest Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

### CUIDADO ACCURACY SOLO DE LAS ENTINDADES, NO DE LA ETIQUETA OUT
## NO USAR EPOCHS SINO BASANDO EN LOSS O ACCURACY PARA TERMINAR LOS EPOCHS


Epoch 1/10
128/128 [==============================] - 25s 149ms/step - loss: 0.2542 - accuracy: 0.9479 - val_loss: 0.0822 - val_accuracy: 0.9726
Epoch 2/10
128/128 [==============================] - 25s 149ms/step - loss: 0.2542 - accuracy: 0.9479 - val_loss: 0.0822 - val_accuracy: 0.9726
Epoch 2/10
128/128 [==============================] - 19s 147ms/step - loss: 0.0702 - accuracy: 0.9748 - val_loss: 0.0666 - val_accuracy: 0.9776
Epoch 3/10
128/128 [==============================] - 19s 147ms/step - loss: 0.0702 - accuracy: 0.9748 - val_loss: 0.0666 - val_accuracy: 0.9776
Epoch 3/10
128/128 [==============================] - 18s 137ms/step - loss: 0.0513 - accuracy: 0.9815 - val_loss: 0.0555 - val_accuracy: 0.9834
Epoch 4/10
128/128 [==============================] - 18s 137ms/step - loss: 0.0513 - accuracy: 0.9815 - val_loss: 0.0555 - val_accuracy: 0.9834
Epoch 4/10
128/128 [==============================] - 17s 136ms/step - loss: 0.0367 - accuracy: 0.9877 - val_loss: 0.0448 - val_ac

In [ ]:
preds = model.predict(test_pad_sequences)

28/28 [==============================] - 1s 39ms/step


In [ ]:
preds.shape

(885, 46, 119)

In [ ]:
def preds_to_index(preds, seq_lens):
  '''
  Turn predictions to numerical indexes, flatten the sentences and discard padding.
  '''
  idx_preds = []
  for pred, seq_len in zip(preds,seq_lens):
      for l in range(seq_len):
        idx_preds.append(np.argmax(pred[l]))
  return idx_preds



Com ja sabeu, NER es una tasca on les dades estan molt desbalancejades. La gran majoria de les nostres etiquetes serán 'O' (outside). Es pot donar el cas on el model tingui una accuracy molt alta predint sempre 'O'.

Per mesurar millor com de bó és el nostre model, calcularem la F1 score per cada classe, així com la mitjana (macro average). Quan prepareu el vostre document, heu de reportar aquesta mitjana com a mètrica del vostre model, no l'acuracy.

In [ ]:
test_labels_idx = preds_to_index(test_labels_one_hot, test_sequence_lengths)
preds_idx = preds_to_index(preds, test_sequence_lengths)

print(classification_report(test_labels_idx, preds_idx, zero_division=1.0))


## SI PRECISION = 1, RECALL Y F1-SCORE = 0 ==> MAL ==> PORQUE NUNCA LO PREDICE, POR TANTO, NUNCA LO PREDECIRÀ BIEN

              precision    recall  f1-score   support

           0       1.00      0.75      0.86        24
           1       1.00      0.79      0.88        29
           2       0.98      1.00      0.99        79
           3       1.00      0.00      0.00         8
           4       0.64      0.37      0.47        19
           5       1.00      0.00      0.00         1
           6       0.75      0.43      0.55         7
           7       0.50      0.33      0.40         3
           8       1.00      0.33      0.50         3
          10       0.00      1.00      0.00         0
          12       0.67      1.00      0.80         2
          13       1.00      0.00      0.00         3
          14       0.65      0.85      0.73        13
          15       0.93      0.93      0.93        14
          16       0.76      0.65      0.70        43
          17       0.86      1.00      0.92        24
          18       1.00      1.00      1.00         6
          19       1.00    

A continuació podeu veure algunes prediccions del model.

In [ ]:
## Modificar Classification report per veure les entitats i no índex  ++ treure <pad> y O. 



In [ ]:
probs = model.predict(test_pad_sequences)
_predicted_labels = np.argmax(probs, axis=2)

predicted_labels = [list(label_encoder.inverse_transform(label)) for label in _predicted_labels]

for i in range(0, 10):
    print('Sentence: ', test_sentences[i])
    print('Original label: ', test_labels[i])
    print('Predicted label: ', predicted_labels[i])
    print()

28/28 [==============================] - 1s 39ms/step
Sentence:  i would like to find a flight from charlotte to las vegas that makes a stop in st. louis
Original label:   O O O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O O O O O B-stoploc.city_name I-stoploc.city_name
Predicted label:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-fromloc.city_name', 'I-toloc.city_name', 'O', 'O', 'O', 'O', 'O', 'B-stoploc.city_name', 'I-stoploc.city_name', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Sentence:  on april first i need a ticket from tacoma to san jose departing before 7 am
Original label:   O B-depart_date.month_name B-depart_date.day_number O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name O B-depart_time.time_relative B-depart_time.time I-depart_time.time
Predicted label:  ['O', 'B-depart_date.month_name', 'B-class_type', 'O',

---

 <h1><a name="ex-six"><center> ✏ Exercici 6 ✏ </a></h1>

Modifiqueu els següents paràmetres del model anterior i analitzeu com afecten a la seva *accuracy*:

 1. **Mida dels Embeddings.** Proveu diferents mides d'*Embeddings* i observeu com canvia l'*accuracy* del model. Heu d'explicar les vostres conclusions.

 2. **Xarxes Convolucionals.** Afegiu capes convolucionals al vostre model. Expliqueu amb detall els valors que heu provat i la vostra motivació a l'hora d'escollir-los. Recordeu, que també podeu provar diferents configuracions de *pooling*.

 3. **Xarxes Recurrents.**  Afegiu capes recurrents al vostre model (LSTM, GRU). Expliqueu amb detall els valors que heu provat i la vostra motivació.

 4. ** Transformer.** Afegiu blocs de Transformer al vostre model. Expliqueu amb detall els valors que heu provat i la vostra motivació.

 5. **Regularització.** Quan proveu configuracions amb més paràmetres veureu que el model comença a tenir *overfitting* molt prompte durant l'entrenament. Afegiu *Dropout* al vostre model. Heu d'explicar la vostra decisió de valors i de posició dins de la xarxa.


 6. **Balancejat de les classes.** Si analitzeu el dataset, veureu que la freqüència de les classes està molt desbalancejada. Keras us permet afegir un pes per a cada classe a l'hora de calcular la loss (Mireu el paràmetre "class_weigth" a la documentació https://keras.io/api/models/model_training_apis/). Calculeu un pes per a cada classe i afegiu-lo al mètode fit del vostre model.

 ---

CONVOLUCIONALS AL INICI DE TRANSFORMER PER EXTREURE MILLORS FEATURES. PERO NO FUNCIONARA PORQUE TRANFORMER ES MAS EFICIENTE CON SEQ LLARGS. O CONVOLUCIÓ QUE SUBSTITUEIXI ELS ATTENTIONS ==> CONFORMERS (QUE NO FUNCIONARÁ PERQUÈ LA FREQUÈNCIA DE MOSTREIG ÉS MOLT BAIXA A DIFERÈNCIA D'UN AUDIO O IMATGE)

EMB + RECURRENT + POSITIONAL + TRANFORMER (NOMÉS 1 HEAD SINO NO TIENE SENTIDO AÑADIR RECURRENT PORQUE ATTENTIONS YA SE LOS COME)

MILLOR MODEL PER 2.1 ES CONVOLUCIONALS + RECURRENTS (TINDRÀ COM INPUT LA SEQ DE CORRE QUE SURT DE CONVOLUCIONALS)
TRANSFORMER FUSIONA CONVOLUCIONALS (SELF-ATTENTION, MULTIHEAD ==> MULTIPLES CONVOLUCIONALS QUE EXPORTEN VECTORS D'IMPORTÀNCIA) I RECURRENTS

## Grid Search con Transformer

In [ ]:
'''# Grid Search para optimizar hiperparámetros del Transformer

from itertools import product
from sklearn.metrics import f1_score
import time

# Definir rangos de hiperparámetros
param_ranges = {
    'embed_dim': [256, 500],
    'num_heads': [1, 2, 4],  # Máximo 5, pero probamos 1,2,4
    'ff_dim': [128, 256, 512],
    'dropout_rate': [0.1, 0.2, 0.3],
    'batch_size': [16, 32, 64],
    'epochs': [10]  # Reducido para acelerar búsqueda
}

# Generar todas las combinaciones posibles
param_combinations = list(product(
    param_ranges['embed_dim'],
    param_ranges['num_heads'],
    param_ranges['ff_dim'],
    param_ranges['dropout_rate'],
    param_ranges['batch_size'],
    param_ranges['epochs']
))

# Filtrar combinaciones inválidas (embed_dim debe ser divisible por num_heads)
valid_combinations = [
    combo for combo in param_combinations 
    if combo[0] % combo[1] == 0  # embed_dim % num_heads == 0
]

total_combinations = len(valid_combinations)

# Almacenar resultados
results = []

print("=" * 80)
print("🔍 GRID SEARCH - Optimización de Hiperparámetros del Transformer")
print("=" * 80)
print(f"\nTotal de combinaciones válidas a probar: {total_combinations}")
print(f"Espacio de búsqueda: {param_ranges}\n")
print(f"⚠️  Nota: Este proceso puede tardar bastante tiempo...")
print(f"    Estimación: ~{total_combinations * 2} minutos (aprox.)\n")

for idx, (embed_dim, num_heads, ff_dim, dropout_rate, batch_size, epochs) in enumerate(valid_combinations, 1):
    print(f"\n{'='*80}")
    print(f"🔄 COMBINACIÓN {idx}/{total_combinations}")
    print(f"{'='*80}")
    
    print(f"\n📋 Hiperparámetros:")
    print(f"  - embed_dim: {embed_dim}")
    print(f"  - num_heads: {num_heads}")
    print(f"  - ff_dim: {ff_dim}")
    print(f"  - dropout_rate: {dropout_rate}")
    print(f"  - batch_size: {batch_size}")
    print(f"  - epochs: {epochs}")
    
    try:
        # Construir modelo con hiperparámetros actuales
        start_time = time.time()
        
        model = Sequential([
            TokenAndPositionEmbedding(
                maxlen=max_sequence_length, 
                vocab_size=len(tokenizer.word_index) + 1, 
                embed_dim=embed_dim
            ),
            TransformerBlock(
                embed_dim=embed_dim, 
                num_heads=num_heads, 
                ff_dim=ff_dim, 
                rate=dropout_rate
            ),
            keras.layers.Dense(len(label_encoder.classes_), activation="softmax")
        ])
        
        model.compile(
            loss="categorical_crossentropy", 
            optimizer="adam", 
            metrics=["accuracy"]
        )
        
        # Entrenar modelo (sin verbose para limpieza)
        print(f"\n⏳ Entrenando modelo...")
        history = model.fit(
            train_pad_sequences, 
            train_labels_one_hot,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(val_pad_sequences, val_labels_one_hot),
            verbose=0  # Sin output durante entrenamiento
        )
        
        # Evaluar en validation set
        val_loss, val_acc = model.evaluate(
            val_pad_sequences, 
            val_labels_one_hot, 
            batch_size=batch_size,
            verbose=0
        )
        
        # Calcular F1-score (macro) en validation
        val_preds = model.predict(val_pad_sequences, verbose=0)
        val_preds_idx = preds_to_index(val_preds, val_sequence_lengths)
        val_labels_idx = preds_to_index(val_labels_one_hot, val_sequence_lengths)
        val_f1_macro = f1_score(val_labels_idx, val_preds_idx, average='macro', zero_division=1.0)
        
        # Evaluar en test set
        test_loss, test_acc = model.evaluate(
            test_pad_sequences, 
            test_labels_one_hot, 
            batch_size=batch_size,
            verbose=0
        )
        
        # Calcular F1-score (macro) en test
        test_preds = model.predict(test_pad_sequences, verbose=0)
        test_preds_idx = preds_to_index(test_preds, test_sequence_lengths)
        test_labels_idx = preds_to_index(test_labels_one_hot, test_sequence_lengths)
        test_f1_macro = f1_score(test_labels_idx, test_preds_idx, average='macro', zero_division=1.0)
        
        training_time = time.time() - start_time
        
        # Guardar resultados
        result = {
            'combination': idx,
            'embed_dim': embed_dim,
            'num_heads': num_heads,
            'ff_dim': ff_dim,
            'dropout_rate': dropout_rate,
            'batch_size': batch_size,
            'epochs': epochs,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'val_f1_macro': val_f1_macro,
            'test_loss': test_loss,
            'test_accuracy': test_acc,
            'test_f1_macro': test_f1_macro,
            'training_time': training_time
        }
        results.append(result)
        
        # Mostrar resultados de esta combinación
        print(f"\n✅ Resultados:")
        print(f"  📊 Val F1-Macro: {val_f1_macro:.4f} | Val Accuracy: {val_acc:.4f}")
        print(f"  📊 Test F1-Macro: {test_f1_macro:.4f} | Test Accuracy: {test_acc:.4f}")
        print(f"  ⏱️  Tiempo: {training_time:.2f}s")
        
        # Guardar resultados parciales cada 5 iteraciones
        if idx % 5 == 0:
            temp_df = pd.DataFrame(results)
            temp_df.to_csv('transformer_grid_search_partial.csv', index=False)
            print(f"  💾 Guardado parcial en 'transformer_grid_search_partial.csv'")
        
    except Exception as e:
        print(f"\n❌ Error en combinación {idx}: {str(e)}")
        continue

print(f"\n\n{'='*80}")
print("📊 RESUMEN DE RESULTADOS - GRID SEARCH COMPLETADO")
print(f"{'='*80}\n")

# Convertir a DataFrame para análisis
results_df = pd.DataFrame(results)

# Ordenar por Val F1-Macro (descendente) - métrica correcta para selección
results_df_sorted = results_df.sort_values('val_f1_macro', ascending=False)

print(f"✅ Total de combinaciones evaluadas: {len(results_df)}/{total_combinations}")
print(f"\n🏆 Top 5 configuraciones por Val F1-Macro:")
print(results_df_sorted[['combination', 'embed_dim', 'num_heads', 'ff_dim', 
                          'dropout_rate', 'val_f1_macro', 'test_f1_macro']].head())

# Mejor configuración según validación
best_config = results_df_sorted.iloc[0]
print(f"\n\n🌟 MEJOR CONFIGURACIÓN (según Val F1-Macro):")
print(f"{'='*80}")
print(f"Combinación: {int(best_config['combination'])}")
print(f"Embed Dim: {int(best_config['embed_dim'])}")
print(f"Num Heads: {int(best_config['num_heads'])}")
print(f"FF Dim: {int(best_config['ff_dim'])}")
print(f"Dropout Rate: {best_config['dropout_rate']:.2f}")
print(f"Batch Size: {int(best_config['batch_size'])}")
print(f"Epochs: {int(best_config['epochs'])}")
print(f"\n📈 Métricas:")
print(f"  - Val F1-Macro: {best_config['val_f1_macro']:.4f} ⭐")
print(f"  - Val Accuracy: {best_config['val_accuracy']:.4f}")
print(f"  - Test F1-Macro: {best_config['test_f1_macro']:.4f}")
print(f"  - Test Accuracy: {best_config['test_accuracy']:.4f}")
print(f"  - Training Time: {best_config['training_time']:.2f}s")
print(f"{'='*80}")

# Guardar resultados completos
results_df_sorted.to_csv('transformer_grid_search_results.csv', index=False)
print(f"\n💾 Resultados completos guardados en 'transformer_grid_search_results.csv'")

print(f"\n💡 Nota: La mejor configuración se selecciona según Val F1-Macro")
print(f"   (no Test F1-Macro, para evitar overfitting en test)")
'''

In [ ]:
'''# Entrenar el mejor modelo encontrado con más epochs
print("=" * 80)
print("🎯 ENTRENANDO MEJOR MODELO CON CONFIGURACIÓN ÓPTIMA")
print("=" * 80)

# Usar los mejores hiperparámetros
best_embed_dim = int(best_config['embed_dim'])
best_num_heads = int(best_config['num_heads'])
best_ff_dim = int(best_config['ff_dim'])
best_dropout = best_config['dropout_rate']
best_batch_size = int(best_config['batch_size'])

print(f"\n📋 Configuración óptima:")
print(f"  - embed_dim: {best_embed_dim}")
print(f"  - num_heads: {best_num_heads}")
print(f"  - ff_dim: {best_ff_dim}")
print(f"  - dropout_rate: {best_dropout}")
print(f"  - batch_size: {best_batch_size}")
print(f"  - epochs: 15 (extendido para mejor entrenamiento)")

# Construir modelo final
best_model = Sequential([
    TokenAndPositionEmbedding(
        maxlen=max_sequence_length, 
        vocab_size=len(tokenizer.word_index) + 1, 
        embed_dim=best_embed_dim
    ),
    TransformerBlock(
        embed_dim=best_embed_dim, 
        num_heads=best_num_heads, 
        ff_dim=best_ff_dim, 
        rate=best_dropout
    ),
    keras.layers.Dense(len(label_encoder.classes_), activation="softmax")
])

best_model.compile(
    loss="categorical_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy"]
)

# Entrenar con más epochs
print("\n⏳ Entrenando modelo final...")
final_history = best_model.fit(
    train_pad_sequences, 
    train_labels_one_hot,
    batch_size=best_batch_size,
    epochs=15,
    validation_data=(val_pad_sequences, val_labels_one_hot),
    verbose=1
)

# Evaluación final en test
print("\n" + "=" * 80)
print("📊 EVALUACIÓN FINAL EN TEST SET")
print("=" * 80)

test_loss, test_acc = best_model.evaluate(
    test_pad_sequences, 
    test_labels_one_hot, 
    batch_size=best_batch_size
)

# Predicciones
preds_final = best_model.predict(test_pad_sequences)
preds_idx_final = preds_to_index(preds_final, test_sequence_lengths)
test_labels_idx_final = preds_to_index(test_labels_one_hot, test_sequence_lengths)

# F1-score macro
f1_macro_final = f1_score(test_labels_idx_final, preds_idx_final, average='macro', zero_division=1.0)

print(f"\n✅ Métricas finales:")
print(f"  - Test Loss: {test_loss:.4f}")
print(f"  - Test Accuracy: {test_acc:.4f}")
print(f"  - Test F1-Macro: {f1_macro_final:.4f}")

# Classification report completo
print("\n" + "=" * 80)
print("📋 CLASSIFICATION REPORT DETALLADO")
print("=" * 80 + "\n")

print(classification_report(
    test_labels_idx_final, 
    preds_idx_final, 
    target_names=label_encoder.classes_,
    zero_division=1.0
))

print("\n💡 Nota: Recuerda que la métrica clave es F1-Macro, no Accuracy")
print("   (debido al desbalanceo de clases en NER)")
'''

**Recurrente + Transformer**


In [ ]:
# ============================================================
# MODELO HÍBRIDO: EMB + RECURRENT + POSITIONAL + TRANSFORMER
# ============================================================
# Arquitectura: Embedding → LSTM Bidireccional → Positional Encoding → Transformer (1 head)
# Rationale: 
#   - LSTM captura dependencias secuenciales locales
#   - Transformer con 1 head captura dependencias globales sin solapar con LSTM
#   - Más de 1 head no tiene sentido porque las multi-head attentions "comerían" 
#     la información ya procesada por la capa recurrente

import tensorflow as tf
import keras

print("=" * 80)
print("🔧 CONSTRUYENDO MODELO HÍBRIDO: LSTM + POSITIONAL + TRANSFORMER")
print("=" * 80)

# Hiperparámetros
embedding_dim = 256
lstm_units = 128  # Unidades LSTM
num_heads = 1  # Solo 1 head para no solapar con LSTM
ff_dim = 256
dropout_rate = 0.2
batch_size = 32
epochs = 10

vocab_size = len(tokenizer.word_index) + 1

print(f"\n📋 Configuración del modelo:")
print(f"  - Embedding Dim: {embedding_dim}")
print(f"  - LSTM Units: {lstm_units}")
print(f"  - Num Heads: {num_heads} (para no solapar con LSTM)")
print(f"  - FF Dim: {ff_dim}")
print(f"  - Dropout Rate: {dropout_rate}")
print(f"  - Vocab Size: {vocab_size}")
print(f"  - Max Sequence Length: {max_sequence_length}")

# Definir capa personalizada para añadir Positional Encoding
class AddPositionalEncoding(keras.layers.Layer):
    """
    Añade codificación posicional a los embeddings de salida del LSTM.
    Similar a TokenAndPositionEmbedding pero solo añade la parte posicional.
    """
    def __init__(self, maxlen, embed_dim):
        super().__init__()
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        
    def call(self, inputs):
        maxlen = tf.shape(inputs)[1]  # Longitud de secuencia
        positions = tf.range(start=0, limit=maxlen, delta=1)
        position_embeddings = self.pos_emb(positions)
        return inputs + position_embeddings

# Construir modelo
print("\n⚙️  Construyendo arquitectura...")

model_hybrid = Sequential([
    # 1. Embedding layer (sin positional encoding aún)
    keras.layers.Embedding(
        input_dim=vocab_size, 
        output_dim=embedding_dim, 
        mask_zero=True,  # Ignorar padding
        name="embedding"
    ),
    
    # 2. LSTM Bidireccional (captura dependencias locales)
    Bidirectional(
        LSTM(lstm_units, return_sequences=True, dropout=dropout_rate),
        name="bidirectional_lstm"
    ),
    
    # 3. Añadir Positional Encoding después del LSTM
    AddPositionalEncoding(maxlen=max_sequence_length, embed_dim=lstm_units*2),
    
    # 4. Transformer Block (1 head para captura global sin solapar)
    TransformerBlock(
        embed_dim=lstm_units*2,  # Bidirectional LSTM duplica dimensión
        num_heads=num_heads, 
        ff_dim=ff_dim, 
        rate=dropout_rate
    ),
    
    # 5. Capa de salida (clasificación por token)
    keras.layers.Dense(len(label_encoder.classes_), activation="softmax", name="output")
])

# Compilar modelo
model_hybrid.compile(
    loss="categorical_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy"]
)

print("\n✅ Arquitectura construida:")
model_hybrid.summary()

# Entrenar modelo
print("\n" + "=" * 80)
print("🚀 ENTRENAMIENTO DEL MODELO HÍBRIDO")
print("=" * 80)

history_hybrid = model_hybrid.fit(
    train_pad_sequences, 
    train_labels_one_hot, 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_data=(val_pad_sequences, val_labels_one_hot),
    verbose=1
)

# Evaluación en test set
print("\n" + "=" * 80)
print("📊 EVALUACIÓN EN TEST SET")
print("=" * 80)

loss_hybrid, accuracy_hybrid = model_hybrid.evaluate(
    test_pad_sequences, 
    test_labels_one_hot, 
    batch_size=batch_size
)

print(f"\n✅ Resultados en Test:")
print(f"  - Test Loss: {loss_hybrid:.4f}")
print(f"  - Test Accuracy: {accuracy_hybrid:.4f}")

# Calcular F1-Score (métrica correcta para NER)
from sklearn.metrics import f1_score, classification_report

preds_hybrid = model_hybrid.predict(test_pad_sequences)
preds_idx_hybrid = preds_to_index(preds_hybrid, test_sequence_lengths)
test_labels_idx_hybrid = preds_to_index(test_labels_one_hot, test_sequence_lengths)

f1_macro_hybrid = f1_score(
    test_labels_idx_hybrid, 
    preds_idx_hybrid, 
    average='macro', 
    zero_division=1.0
)

print(f"  - Test F1-Macro: {f1_macro_hybrid:.4f} ⭐")

# Classification report detallado
print("\n" + "=" * 80)
print("📋 CLASSIFICATION REPORT DETALLADO")
print("=" * 80 + "\n")

print(classification_report(
    test_labels_idx_hybrid, 
    preds_idx_hybrid, 
    target_names=label_encoder.classes_,
    zero_division=1.0
))

print("\n💡 Arquitectura Híbrida: LSTM captura patrones locales, Transformer (1 head)")
print("   captura dependencias globales sin redundancia con el LSTM.")


---

<h1><a name="section-four"> 4. Lliurable </a></h1>

Heu d'entregar un document PDF de com a **màxim 10 pàgines** que incloga els resultats de tots els exercicis així com una explicació de cadascun dels resultats i de la modificació que heu fet. L'estructura del document és:

1. Introducció.
2. Experiments i Resultats (amb raonament).
3. Conclusions.

No cal que afegiu el vostre codi al document, podeu entregar el *notebook* juntament amb el document.

 ---